In [295]:
import numpy as np
f = open("capture.out", "r"); next(f); g = open("capture.out.list.txt", "r"); 
camN = int(f.readline().split(' ')[0]); F_K12s=[]; Rs = []; Ts = []; filenames = []; 
for i in range( camN ):    # まずはf（焦点距離）, k1, k2（歪みパラメータ）
    F_K12s.append( [ float(x) for x in f.readline().strip().split(' ')] )
    R = np.array([ [ float(x) for x in f.readline().strip().split(' ')], # R（回転行列）
                          [ float(x) for x in f.readline().strip().split(' ')],
                          [ float(x) for x in f.readline().strip().split(' ')]  ])
    R[:,1:] = -R[:,1:]; Rs.append( R )  # Z方向を反転（鏡像にならないようYも反転） 
    Ts.append( [ float(x) for x in f.readline().strip().split(' ')] ) # T（並進ベクトル）
    filenames.append( g.readline().strip() ) #  画像ファイル名も読み込む
f.close

import cv2
imgs = []
for i in range( camN ):#for i in range( 1 ):
    img = cv2.imread("images/"+filenames[i] ); h, w = img.shape[:2]
    cameraMatrix1 = np.array([  # カメラ内部行列
            [F_K12s[i][0],  0.0,                w/2.0 ],
            [0.0,                F_K12s[i][0],  h/2.0  ],
            [0.0,                0.0,                1.0]    ]  )
    distCoeffs1 = np.array( [ F_K12s[ i ][1], F_K12s[ i ][ 2 ], 0, 0, 0 ] ) # 歪み係数
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix( 
             cameraMatrix1, distCoeffs1, (w,h), 1, (w,h) )
    map = cv2.initUndistortRectifyMap( cameraMatrix1,   # カメラ行列
                                                            distCoeffs1,        #  歪み係数
                                                            np.array( Rs[i] ),  # 回転行列   
                                                            newcameramtx, (w,h),  cv2.CV_32FC1) 
    dst = cv2.remap( img, map[0],  map[1], cv2.INTER_LINEAR)
    cv2.imwrite('corrected/'+filenames[ i ], dst ); imgs.append( dst )
    
#plt.imshow( dst )
# わかりやすい
# http://russeng.hatenablog.jp/entry/2016/03/06/190901
# Bundler.out についてはこちら
# http://www.am.sanken.osaka-u.ac.jp/~mitsugami/note/bundler/
import math
start = -3.0; end = 3.0; n = 100;  # 撮影位置のXY限界
posn = [ [0 for i in range(n)] for j in range(n) ]# [x,y,0]位置のカメラID配列
for iy in range( n ):  
    for ix in range( n ):
        y = start + iy * (end - start ) / n; x = start + ix * (end - start ) / n
        nearCam = 0; nearDistance = 100
        for i in range( camN ):                    
            distance = math.sqrt( (Ts[i][0] - x)**2 + (Ts[i][1]-y)**2 )
            if (distance < nearDistance):
                nearCam = i; nearDistance = distance
        posn[ix][iy] = nearCam

def nearCam(x, y):         # (x,y)座標に近いカメラIDを返す(z座標は０)
    ix = int( (x - start) / ( (end - start ) / n ) )
    iy = int( (y - start) / ( (end - start ) / n ) )
    return posn[ ix ][ iy ] # （上部で）事前作成の配列でIDを返す

def cam_IDfromXYandUV(x, y, z, u, v): #   仮想カメラ位置[X,Y,Z]と画素位置[U,V]から実カメラ位置XY算出
    diffX = (u - w/2.0) / F_K12s[0][0] * z  # 焦点距離はカメラ0のものを使う 
    diffY = (v - h/2.0)  / F_K12s[0][0] * z  # 焦点距離はカメラ0のものを使う
    return nearCam( x-diffX, y+diffY )     # Y軸は（vが下向きなので）反転不要

npimg = np.zeros((h, w, 3))
for y in range( h ):
    for x in range( w ):
        r = np.random.normal(0, 0.6)  # 擬似輪郭の低減にバラツキ付与
        i = cam_IDfromXYandUV( -0.5, -0.5, 2+r, x, y)
        if ( y < imgs[ i ].shape[0] and x < imgs[ i ].shape[1] ):
            npimg[y, x] = imgs[ i ][y, x]
plt.imshow( npimg/255 )
cv2.imwrite('result.png', npimg )